This The implementation of the S-DES algorithm:


1. P-BOX:


In [267]:
initial_p_box_mapping = [2,6,3,1,4,8,5,7]
final_p_box_mapping = [4,1,3,5,7,2,8,6]
compression_p_box_mapping = [6,3,7,4,8,5,10,9]
straight_p_box_mapping = [3,5,2,7,4,10,1,9,8,6]
straight_p_box_mapping_small = [2,4,3,1]
expansion_p_box_mapping = [4,1,2,3,2,3,4,1]
def generalBox(data,mapping):
    return [data[i - 1] for i in mapping]

2. Key generation:


In [268]:
def leftShift(data,numberOfShifts):
    for shifts in range(numberOfShifts):
        data.append(data.pop(0))
    return data

def keyGeneration(key):
    keys = []
    data = [*str(key)]
    data = generalBox(data,straight_p_box_mapping)
    
    k1 = data[:5]
    k2 = data[5:]
    
    k1 = leftShift(k1, 1)
    k2 = leftShift(k2, 1)
    
    keys.append(generalBox(k1+k2,compression_p_box_mapping))
    
    k1 = leftShift(k1, 2)
    k2 = leftShift(k2, 2)
    
    keys.append(generalBox(k1+k2,compression_p_box_mapping))
    
    return keys

3. The S-DES round:


In [269]:
one_s_box_mapping = [["01", "00", "11", "10"], ["11", "10", "01", "00"], [
    "00", "10", "01", "11"], ["11", "01", "11", "10"]]
two_s_box_mapping = [["00", "01", "10", "11"], ["10", "00", "01", "11"], [
    "11", "00", "01", "00"], ["10", "01", "00", "11"]]


def sBox_permutation(data, mapping):
    i = int('00'+(data[0]) + (data[3]), 2)
    j = int('00'+(data[1]) + (data[2]), 2)
    return mapping[i][j]


def function(r, key):

    r = generalBox(r, expansion_p_box_mapping)

    xor_result = [str((ord(a) ^ ord(b))) for a, b in zip(r, key)]
    
    k1 = xor_result[:4]
    k2 = xor_result[4:]
    
    sBoxResult = sBox_permutation(
        k1, one_s_box_mapping) + sBox_permutation(k2, two_s_box_mapping)
        
    return generalBox([*str(sBoxResult)], straight_p_box_mapping_small)


def mixer(l, r, key):

    function_result = function(r, key)
    
    xor_result = [str((ord(a) ^ ord(b))) for a, b in zip(l, function_result)]

    return [xor_result, r]


def Round(l, r, key, withSwapper):
    mixerResult = mixer(l, r, key)
    if(withSwapper):
        return [mixerResult[1], mixerResult[0]]
    else:
        return mixerResult


4. Compine everything to encrpyt and decrypt:

In [270]:
def encrypt(data,key):
    print('Start Encrypting')
    initial_permutation = generalBox(data,initial_p_box_mapping)
    keys = keyGeneration(key)
    round_result_one = Round(initial_permutation[:4],initial_permutation[4:],keys[0],True)
    round_result_two = Round(round_result_one[0],round_result_one[1],keys[1],False)
    return generalBox(round_result_two[0]+round_result_two[1],final_p_box_mapping)

def decrypt(data,key):
    print('Start Encrypting')
    initial_permutation = generalBox(data,initial_p_box_mapping)
    keys = keyGeneration(key)
    round_result_one = Round(initial_permutation[:4],initial_permutation[4:],keys[1],True)
    round_result_two = Round(round_result_one[0],round_result_one[1],keys[0],False)
    return generalBox(round_result_two[0]+round_result_two[1],final_p_box_mapping)


print(encrypt("11110010","1011100110"))
print(decrypt("01110111","1011100110"))

Start Encrypting
['0', '1', '1', '1', '0', '1', '1', '1']
Start Encrypting
['1', '1', '1', '1', '0', '0', '1', '0']
